<a href="https://colab.research.google.com/github/stefan76er/C.alg/blob/master/PremData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
standings_url = "https://fbref.com/en/comps/9/Preimier-League-Stats"


In [ ]:
data = requests.get(standings_url)

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

IndexError: list index out of range

In [ ]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

['https://fbref.com/en/squads/19538871/2023-2024/matchlogs/all_comps/shooting/Manchester-United-Match-Logs-All-Competitions',
 'https://fbref.com/en/squads/19538871/2023-2024/matchlogs/all_comps/shooting/Manchester-United-Match-Logs-All-Competitions',
 'https://fbref.com/en/squads/19538871/2023-2024/matchlogs/all_comps/shooting/Manchester-United-Match-Logs-All-Competitions',
 'https://fbref.com/en/squads/19538871/2023-2024/matchlogs/all_comps/shooting/Manchester-United-Match-Logs-All-Competitions']

In [ ]:
team_url = team_urls[0]

In [ ]:
data = requests.get(team_url)

In [ ]:
import pandas as pd

In [ ]:
matches = pd.read_html(data.text, match="Scores & Fixtures")

ValueError: No tables found

In [ ]:
matches[0]

KeyError: 0

In [ ]:
soup = BeautifulSoup(data.text)

In [ ]:
links = soup.find_all('a')

In [ ]:
links = [l.get("href") for l in links]

In [ ]:
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [ ]:
links

[]

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

IndexError: list index out of range

In [ ]:
shooting = pd.read_html(data.text, match="Shooting")[0]

ValueError: No tables found

In [ ]:
shooting.columns = shooting.columns.droplevel()

ValueError: Cannot remove 1 levels from an index with 1 levels: at least one level must be left.

In [ ]:
shooting.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Match Report
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,13.9,0.0,0,0,1.9,1.9,0.12,1.1,1.1,Match Report
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Match Report
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle United,...,17.9,0.0,0,0,1.0,1.0,0.07,0.0,0.0,Match Report
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield United,...,17.3,2.0,0,1,3.5,2.8,0.10,-1.5,-0.8,Match Report


In [ ]:
team_data = matches[0].merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]], on="Date")

In [ ]:
team_data

In [ ]:
matches[0].shape

In [ ]:
shooting.shape

In [ ]:
years = list(range(2023,2010, -1))

In [ ]:
years

In [ ]:
all_matches = []

In [ ]:
standings_urls = "https://fbref.com/en/comps/9/Preimier-League-Stats"

In [ ]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"

    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]

        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)